In [ ]:
%pylab
%matplotlib inline
import skimage as sk
import skimage.io
import skvideo
import skvideo.io
import torch
import torch as T
import scipy
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import normalize
from tqdm import tqdm
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma)

In [ ]:
directory = "/home/ubuntu/"
# filename = "20180410_1.mp4"
filename = "20180109_2.mp4"
vid_gen = skvideo.io.vreader(directory+filename)

In [ ]:
for frame in vid_gen:
    firstFrame = frame
    break

n = 20
frames = np.zeros([n,*frame.shape]).astype(np.uint8)
f = 0
# get some frames over time 
for i,frame in enumerate(vid_gen):
    if f >= n:
        break
    elif i % 1000 == 0:
        frames[f] = frame
        f+=1

vid_gen = skvideo.io.vreader(directory+filename)

In [ ]:
plt.imshow(firstFrame)

In [ ]:
# find a good frame for ROI
plt.imshow(frames[11])

# ROI selection

In [ ]:
pad = 30
maxThreshold = 30
minThreshold = 7

frame = np.full([pad*2+firstFrame.shape[0],pad*2+firstFrame.shape[1]], 1)
frame[pad:-pad,pad:-pad] = denoise_tv_chambolle(frames[11,:,:,0]).copy()*255
frame[frame>maxThreshold] = maxThreshold
frame[frame<minThreshold] = 0
plt.imshow(frame)
plt.colorbar()

In [ ]:
fishCenter = (pad+235,pad+190)
pad = 200

fishROI = frame[fishCenter[0]-pad:fishCenter[0]+pad,fishCenter[1]-pad:fishCenter[1]+pad]

# fishROI = normalize(fishROI, norm='l2')
fishROI = fishROI - fishROI.mean()
# fishROI = (fishROI - fishROI.mean())/fishROI.std()
# fishROI = fishROI - fishROI.sum()/len(fishROI)
fishROI = fishROI.astype(float32)

fishROI[pad,pad] = np.max(fishROI)*2
plt.imshow(fishROI)
plt.colorbar()
imageW, imageH = firstFrame.shape[:-1]

# Filter construction

In [ ]:
def rotateImage(img, angle, pivot):
    padX = [img.shape[1] - pivot[1], pivot[1]]
    padY = [img.shape[0] - pivot[0], pivot[0]]
    imgP = np.pad(img, [padY, padX], 'constant')
    imgR = scipy.ndimage.rotate(imgP, angle, reshape=False)
    return imgR[padY[0] : -padY[1], padX[0] : -padX[1]]

def ROI_to_filters(img, center, pad, imageW, imageH, nfilters=8, fft=False, dtype=np.float32):
    filter_size = 2*pad
    # we will use the center of the img plus a pad to create rotationally invariant filters
    filters = np.zeros([nfilters,filter_size,filter_size]).astype(np.float64)
    norm = (img - img.mean()).astype(np.float64) # subtract mean
    for i in range(nfilters):
        # rotate the image first around the provided pivot
        rot = rotateImage(norm, i * 360 / nfilters, center)
        # select the ROI for the filter
        filters[i] = rot[center[0]-pad:center[0]+pad,center[1]-pad:center[1]+pad]
        
    if fft:
        complex_filters = T.cuda.FloatTensor(nfilters,imageW, imageH, 2).zero_()
        complex_filters[:,0:filter_size,0:filter_size,0] = T.from_numpy(filters)
        return T.fft(complex_filters,2)
    else:
        return T.from_numpy(filters.astype(dtype)).cuda()

In [ ]:
dtype = np.float32
filters = ROI_to_filters(frame, fishCenter, pad, imageW, imageH, 8,dtype=dtype)
filters_fft = ROI_to_filters(frame, fishCenter, pad, imageW, imageH, 8, fft=True)

In [ ]:
plt.subplots(6,6,figsize=(12,12))
for i in range(32):
    plt.subplot(6,6,1+i)
    plt.imshow(filters[i].cpu().numpy().astype(np.float32))

In [ ]:
img = firstFrame[:,:,0].astype(dtype)

# Tracking code

In [ ]:
def heatmap_fft(img, w, shape):
    fimg = T.cuda.FloatTensor(*w.shape[1:]).zero_()
    fimg[:,:,0] = T.from_numpy(img).cuda()
    conv = T.ifft(T.fft(fimg,2)*w,2)
    return conv[:,:,:,0]

def heatmap(img, w, shape):
    fimg = T.from_numpy(img).cuda()
    pad = (int((w.shape[1])/2),int((w.shape[2])/2))
    conv = F.conv2d(fimg[None,None,:,:,], w[:,None],padding=pad)
    return conv[0,:,:,:]


def find_position_fft(img, w, shape):
    fimg = T.cuda.FloatTensor(*w.shape[1:]).zero_()
    fimg[:,:,0] = T.from_numpy(img).cuda()
    conv = T.ifft(T.fft(fimg,2)*w,2)
    idx = conv.argmax()
    return np.unravel_index(idx, shape)[1:3]

def find_position(img, w):
    fimg = T.from_numpy(img).cuda()
    pad = (int((w.shape[1]-1)/2),int((w.shape[2]-1)/2))
    conv = F.conv2d(fimg[None,None,:,:,], w[:,None],padding=pad)
    idx = int(conv.argmax())
    y, x = np.unravel_index(idx, conv.shape)[2:]
    return y, x

def find_position_threshold(img, w, minThreshold=0,maxThreshold=20):
    fimg = T.from_numpy(img).cuda()
    fimg[fimg>maxThreshold] = maxThreshold
    fimg[fimg<minThreshold] = 0
    pad = (int((w.shape[1]-1)/2),int((w.shape[2]-1)/2))
    conv = F.conv2d(fimg[None,None,:,:,], w[:,None],padding=pad)
    idx = int(conv.argmax())
    y, x = np.unravel_index(idx, conv.shape)[2:]
    return y, x


def find_position_fft_np_argmax(img, w, shape):
    fimg = T.cuda.FloatTensor(*w.shape[1:]).zero_()
    fimg[:,:,0] = T.from_numpy(img).cuda()
    conv = T.ifft(T.fft(fimg,2)*w,2)
    idx = conv[:,:,:,0].cpu().numpy().argmax()
    return np.unravel_index(idx, shape)[1:3]

shape = [1, filters.shape[0],*img.shape]

# Benchmarking

In [ ]:
%%timeit -r 3 -n 3
pos = heatmap_fft(img,filters_fft,shape)

In [ ]:
%%timeit -r 3 -n 3
# for some reason, pytorch argmax is performing very slowly.
pos = find_position_fft(img,filters_fft,shape)

In [ ]:
%%timeit -r 3 -n 3
# numpy's argmax is performing much better, but this is suboptimal
# as must transfer entire image back to CPU
pos = find_position_fft_np_argmax(img,filters_fft,shape)

In [ ]:
%%timeit -r 3 -n 3
pos = heatmap(img,filters,shape)

In [ ]:
%%timeit -r 3 -n 3
pos = find_position(img,filters)

In [ ]:
img8 = img.astype(np.uint8)
filters8 = filters.byte()

In [ ]:
def heatmap(img, w, shape):
    fimg = T.from_numpy(img).cuda()
    pad = (int((w.shape[1])/2),int((w.shape[2])/2))
    conv = F.conv2d(fimg[None,None,:,:,], w[:,None])
#     conv = F.conv2d(fimg[None,None,:,:,], w[:,None])
    return conv[0,:,:,:]

class Net(nn.Module):
    def __init__(self,c,kernel):
        super(Net, self).__init__()
        pad = (int((kernel[0])/2),int((kernel[1])/2))
        self.conv2d = nn.Conv2d(1,c,kernel)

    def forward(self, x):
        x = self.conv2d(x[None,None])
        return x[0,:,:,:]

model = Net(8,filters.shape[1:])
model.cuda()
fimg = T.from_numpy(img).cuda()
ITERS = 100

torch.cuda.synchronize()
start = time.time()
for i in range(ITERS):
    model(fimg)
# pos = heatmap(img,filters,shape)
torch.cuda.synchronize()
print("Time / iteration: ", (time.time()-start)/ITERS)

model = Net(8,filters.shape[1:])
model.cuda().half()
fimg = T.from_numpy(img16).cuda()
torch.cuda.synchronize()
start = time.time()
for i in range(ITERS):
    model(fimg)
# pos = heatmap(img16,filters16,shape)
torch.cuda.synchronize()
print("Time / iteration: ", (time.time()-start)/ITERS)

In [ ]:
T.version.cuda

In [ ]:
T.from_numpy(img16).dtype

In [ ]:
%%timeit -r 3 -n 3
pos = heatmap(img16,filters16,shape)

In [ ]:
filters.dtype

In [ ]:
%%timeit -r 3 -n 3
pos = find_position_threshold(img,filters)

# FFT conv validation (failing)

In [ ]:
plt.imshow(heatmap(img, filters, shape).cpu().numpy()[0].astype(np.float64))
plt.colorbar()

In [ ]:
plt.imshow(heatmap_fft(img, filters_fft, shape)[0])
plt.colorbar()

# Create tracking video

In [ ]:
def drawROI(img, y, x, width=20,dot=1):
    colorimg = img.copy()
    colorimg[y-dot:y+dot,x-dot:x+dot] = [0,255,0]

    ystart = max(0,y-width)
    yend = min(img.shape[0]-1,y+width)
    xstart = max(0,x-width)
    xend = min(img.shape[1]-1,x+width)
    colorimg[ystart:yend,xstart] = [255,0,0]
    colorimg[ystart:yend,xend] = [255,0,0]
    colorimg[ystart,xstart:xend] = [255,0,0]
    colorimg[yend,xstart:xend] = [255,0,0]
    return colorimg

In [ ]:
pos = find_position_threshold(img,filters)
plt.figure(figsize=(17,17))
plt.imshow(drawROI(frames[7],*pos,80))

In [ ]:
writer = skvideo.io.FFmpegWriter(directory + "track_close_32filters_" + filename)
for t, frame in tqdm(enumerate(vid_gen)):
    if t % 10 ==0:
        img = frame[:,:,0].astype(np.float32)
        pos = find_position(img,filters)
        f = drawROI(frame,*pos,80)
        writer.writeFrame(f)
    if t > 1000*10:
        break
writer.close()